## <center><strong>Notebook<span style= 'color: #51FCC6'> Describtion

## <center><strong>Importing<span style= 'color: #48E0DC'> Packeges

In [5]:
import time
import sqlite3
import warnings
import requests
import itertools
import cloudscraper
import numpy              as np
import pandas             as pd
import seaborn            as sns
import matplotlib.pyplot  as plt

from itertools                       import count
from bs4                             import BeautifulSoup
from selenium                        import webdriver
from selenium.webdriver.common.by    import By
from selenium.webdriver.common.keys  import Keys
from fake_useragent                  import UserAgent
from IPython.display                 import set_matplotlib_formats
from selenium.webdriver.support.ui   import WebDriverWait
from selenium.webdriver.support      import expected_conditions as EC

In [2]:
COLORS = ['#51fcc6', '#48e0dc', '#5cd3f7', '#4895e0', '#517afc']

%matplotlib inline

NUMERICS = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
            'uint16', 'uint32', 'uint64']
MAX_PAGES = 300

set_matplotlib_formats('pdf', 'svg')
warnings.filterwarnings('ignore')

data_jobs_titles = ['Data entry', 'Data Engineering',
                    'Data scientist', 'Data Analyst',
                    'Machine Learning']

C:\Users\FreeComp\AppData\Local\Temp\ipykernel_17264\725494067.py:9: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('pdf', 'svg')


## <center><strong>Setting up the<span style= 'color: #5CD3F7'> Web scrapers

In [43]:
def scrape_page(url: str) -> BeautifulSoup:

    scraper = cloudscraper.create_scraper(delay= 10,browser= {
        'browser': 'chrome',
        'platform': 'windows',
        'desktop': True,
        'mobile': False}) 
    
    content = scraper.get(url).text
    soup    = BeautifulSoup(content)
    
    return soup

In [55]:
def upwork_scraper(job_title : str,
                   page      : int) -> list:

    url         : str  = f'https://www.upwork.com/search/profiles/?page={page}&q={job_title}'
    soup        : str  = scrape_page(url)
    loaded_jobs : list = soup.find_all('div', class_= 'up-card-section up-card-hover')

    return  loaded_jobs

In [65]:
def guru_scraper(job_title : str,
                 page      : int) -> list:

    url         : str = f'https://www.guru.com/d/freelancers/skill/{job_title}/pg/{page}/'
    soup        : str  = scrape_page(url)
    loaded_jobs : list = soup.find_all('div', class_= 'record record--avatarCheck findGuruRecord')

    return loaded_jobs

Now we will try to calculate how much did each scraper take to collect the data

In [67]:
%timeit upwork_scraper('Data analytics', 1)
%timeit guru_scraper('Data analytics', 1)

1.8 s ± 243 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.03 s ± 153 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## <center><strong>Collecting the Websites <span style = 'color: #4895e0'> HTML</span>

In [25]:
soups = {'platform': [],
         'soup_job_type_': [],
         'soup_job_title': [],
         'soup_page':  [],
         'soup': []}

In [26]:
# If the job type_ is services in upwork the pages paremeter will refer to how many pages 
# will render at once while the other scrapers will render the spesfic spesfied page only.

def get_all_pages_soups(pph_type_, upwork_type_, guru_type_, max_page: int= 999) -> dict:

    print('Strating ...')
    flag = False
    
    for i in count(0):
        
        page = i + 1

        fivver_type_ = 'services'
        freelancer_type_ = 'jobs'

        if flag:
            break

        print(f'Loading page {page} of each website ...')

        for data_job_title in data_jobs_titles:

            lines = [f'freelancer_html = freelancer_scraper({page}, '{data_job_title}')',
                     f'pph_html = pph_scraper({page}, '{data_job_title}', '{pph_type_}')',
                     f'fivver_html = fivver_scraper({page}, '{data_job_title}')']
            # f'guru_html = fivver_scraper({page}, '{data_job_title}')',
            # upwork_html = upwork_scraper({page}, '{data_job_title}')

            if len(lines) < 1 or page > max_page:

                print('Finished ...')
                flag = True

            if flag:
                break
            

            for line_idx, line in enumerate(lines):

                equall_sign_idx = line.find('=') - 1
                first_underscore_idx = line.find('_')

                # if page > max_page:
                #     del lines[line_idx]
                try:
                    exec(line)
                    print(line)
                    
                    soups['soup_job_title'].append(data_job_title)
                    soups['platform'].append(line[:equall_sign_idx])
                    soups['soup_page'].append(page)
                    soups['soup'].append(eval(str(line[:equall_sign_idx])))
                    soups['soup_job_type'].append(eval(f'{line[:first_underscore_idx]}_type'))


                except KeyError as e:

                    print(e)
                    del lines[line_idx]

flag = False

for data_job_title in data_job_titles:

    if flag:
        break

    try:
        soups['soup_job_title'].append(data_job_title)
        soups['soup_name'].append('upwork_html')
        soups['soup_page'].append(MAX_PAGE)
        soups['soup'].append(upwork_scraper(MAX_PAGE, data_job_title, 'services'))
        soups['soup_job_type_'].append('services')

    except KeyError as e:
        flag = True
        print(e)

NameError: name 'data_job_titles' is not defined

In [108]:
get_all_pages_soups(pph_type_= 'freelance-jobs', upwork_type_= 'jobs',
                    guru_type_= 'jobs', max_page= MAX_PAGE)

Strating ...
Loading page 1 of each website ...
freelancer_html = freelancer_scraper(1, "data-entry")
pph_html = pph_scraper(1, "data-entry", "freelance-jobs")
fivver_html = fivver_scraper(1, "data-entry")
freelancer_html = freelancer_scraper(1, "data-processing")
pph_html = pph_scraper(1, "data-processing", "freelance-jobs")
fivver_html = fivver_scraper(1, "data-processing")
Loading page 2 of each website ...
freelancer_html = freelancer_scraper(2, "data-entry")
pph_html = pph_scraper(2, "data-entry", "freelance-jobs")
fivver_html = fivver_scraper(2, "data-entry")
freelancer_html = freelancer_scraper(2, "data-processing")
pph_html = pph_scraper(2, "data-processing", "freelance-jobs")
fivver_html = fivver_scraper(2, "data-processing")
Loading page 3 of each website ...
Finished ...


In [109]:
max_length = max(len(lst) for lst in soups.values())
soups = {key: lst + [float('nan')] * (max_length - len(lst)) for key, lst in soups.items()}

soups = pd.DataFrame(soups)

soups.dropna(inplace= True)

In [110]:
print('Printing the title of each page to check if it's loaded correctly.\n')

for soup in soups['soup']:
    print('-  ', soup.title.text.replace('\n', '').replace('\t', ''))

soups = pd.DataFrame(soups)

Printing the title of each page to check if it's loaded correctly.

-   Top Jobs - Freelancer.com
-   Freelance Jobs, Work & Projects in Jul 2023 | PeoplePerHour
-   Data Entry Services Online | Fiverr
-   Top Jobs - Freelancer.com
-   Freelance Jobs, Work & Projects in Jul 2023 | PeoplePerHour
-   Data Processing Services Online | Fiverr
-   Top Jobs - Freelancer.com
-   Freelance Jobs, Work & Projects in Jul 2023 | PeoplePerHour
-   Your Access To This Website Has Been Blocked
-   Top Jobs - Freelancer.com
-   Freelance Jobs, Work & Projects in Jul 2023 | PeoplePerHour
-   Data Processing Services Online | Fiverr


Looks like everything is working as expected now let's start reel work by extracting the HTML tags data from the scraped<br>
pages.<br> <br>

But we will need to collect the data from each one of those pages with unique way further more we need to optimize the<br>
data extractor to collect the data from other pages from the same site like 'jobs' site and 'profiles' sites on upwork.<br>

In [111]:
def get_cards_links(soup_name, tag, link_template,
                    link_attr, tag_class, attr= 'class') -> list:

    soup = soups[soups['soup_name'] == soup_name]['soup'].values[0]

    cards = soup.find_all(tag, attrs= {attr: tag_class})
    cards_ids = []

    if link_attr != '':

        for card in cards:

            card_id = link_template.replace('###', card[link_attr])

            if len(card_id) > len(link_template):
                cards_ids.append(card_id)

        return  cards_ids

    else:
        return  cards

In [131]:
soups = soups.drop(['soup_page'], axis= 1).groupby(
    ['soup_name', 'soup_job_type_', 'soup_job_title'])['soup'].apply(
        lambda x: BeautifulSoup(''.join(str(x)))).reset_index()



In [135]:
soups.head(6)

,soup_name,soup_job_type,soup_job_title,soup
0,fivver_html,services,data-entry,"[[[<p>2 [[[<link href=""https://fiverr-res.c..."
1,fivver_html,services,data-processing,"[[[<p>5 [[[<link data-processing="""" dtype:..."
2,freelancer_html,jobs,data-entry,[[[<p>0 [[[<style>.pac-container{background...
3,freelancer_html,jobs,data-processing,[[[<p>3 [[[<style>.pac-container{background...
4,pph_html,freelance-jobs,data-entry,"[[[<p>1 [[[<meta content=""utf-8"" data-react..."
5,pph_html,freelance-jobs,data-processing,"[[[<p>4 [[[<meta content=""utf-8"" data-reac..."


In [113]:
# We can't collect guru data this wat becuase it has slightly deffrint case in terms
# of collecting the jobs URLs

## --------------------------Upwork--------------------------
upwork_gigs_links = get_cards_links('upwork_html', 'a', 'https://upwork.com###', 'href',
                                        'up-n-link project-tile up-card m-0 p-0 up-card-section up-card-hover project-tile_flexible')

upwork_profiles_links = get_cards_links('upwork_html', 'div', 'https://www.upwork.com/search/profiles/?profile=###', 'data-test-key',
                                         'up-card-section up-card-hover')

upwork_jobs_parents = get_cards_links('upwork_html', 'h3', 'https://www.upwork.com###', '', 'my-0 p-sm-right job-tile-title')
upwork_jobs_links = []

for parent in upwork_jobs_parents:
    upwork_jobs_links.append('https://www.upwork.com' + parent.find('a', class_= 'up-n-link')['href'])
## ----------------------------------------------------------


## --------------------------Fivver--------------------------
fivver_gigs_links = get_cards_links('fivver_html', 'a', 'https://www.fiverr.com###', 'href', 'tbody-5 p-t-4 YLycza2 u9KHmsf')
## ----------------------------------------------------------


## ---------------------People per hour----------------------
pph_services_links = get_cards_links('pph_html', 'a', 'https://www.peopleperhour.com###', 'href', 
                                     'card__title-wrapper⤍FreelancerCard⤚2-NWk')

pph_freelancers_links = get_cards_links('pph_html', 'a', 'https://www.peopleperhour.com###', 'href',
                                        'card__title-wrapper⤍FreelancerCard⤚2-NWk')

pph_jobs_links = get_cards_links('pph_html', 'a', 'https://www.peopleperhour.com###', 'href',
                                 'item__url⤍ListItem⤚20ULx')
## ----------------------------------------------------------


## ------------------------Guru------------------------------
guru_cards_parents_profiles = get_cards_links('guru_html', 'h2', 'https://guru.com###', '',
                                              'jobRecord__title jobRecord__title--changeVisited')

guru_cards_parents_jobs = get_cards_links('guru_html', 'h2', 'https://guru.com###', '',
                                          'jobRecord__title jobRecord__title--changeVisited')
## ----------------------------------------------------------


## ----------------------Freelancer---------------------------
freelancer_links = get_cards_links('freelancer_html', 'a', 'https://www.freelancer.com###', 'href',
                                        'LinkElement ng-star-inserted')

# freelancer_freelancers_links = get_cards_links('freelancer_soup', 'a', 'https://www.freelancer.com###', 'href',
#                                               'LinkElement ng-star-inserted')

In [116]:
fivver_gigs_links

['https://www.fiverr.com/private_finder/do-private-open-source-investigation?context_referrer=subcategory_listing&source=pagination&ref_ctx_id=9e83d865572d42844d83e6a871099c2a&pckg_id=1&pos=1&context_type=rating&funnel=9e83d865572d42844d83e6a871099c2a&imp_id=27d3f213-ff2e-4da8-9cd5-e1732a7f2279',
 'https://www.fiverr.com/excel_buff/do-excel-data-entry-mining-scraping-find-email-address-shopify-product-lists?context_referrer=subcategory_listing&source=pagination&ref_ctx_id=9e83d865572d42844d83e6a871099c2a&pckg_id=1&pos=2&context_type=rating&funnel=9e83d865572d42844d83e6a871099c2a&imp_id=c00a939c-c2f4-4148-aec2-6cd2ef4256a0',
 'https://www.fiverr.com/humairramzan08/do-data-entry-email-collection-data-collection-web-scraper-data-scraper?context_referrer=subcategory_listing&source=pagination&ref_ctx_id=9e83d865572d42844d83e6a871099c2a&pckg_id=1&pos=3&context_type=rating&funnel=9e83d865572d42844d83e6a871099c2a&imp_id=30c43de8-615b-463b-be6b-9e3a4424df2a',
 'https://www.fiverr.com/leadscrape

In [ ]:
def get_upwork_links(job_type_: str) -> list:  

    card_tag = 'a'
    link_template = 'https://www.upwork.com###'
    link_attr = 'href'

    if job_type_ == 'profiles':

        card_tag = 'div'
        card_tag_class = 'up-card-section up-card-hover'
        link_template = 'https://www.upwork.com/search/profiles/?q=profiles&profile=###'
        link_attr = 'data-test-key'
    
    elif job_type_ == 'jobs':
        card_tag_class = 'up-n-link'


    elif job_type_ == 'services':
        card_tag_class = 'up-n-link project-tile up-card m-0 p-0 up-card-section up-card-hover project-tile_flexible'

    else:
        raise Exception('Please input correct work type_ from ['profiles', 'jobs', 'services'].')


    soup = soups[soups['soup_name'] == 'upwork_soup']['soup'][0]

    cards = soup.find_all(card_tag, class_= card_tag_class)
    cards_ids = []

    for card in cards:
        
        card_id = link_template.replace('###', card[link_attr])
        
        if len(card_id) > len(link_template):
            cards_ids.append(card_id)

    return  cards_ids


def get_fivver_links() -> list:

    card_tag = 'div'
    card_tag_class = 'basic-gig-card'
    link_template = 'https://www.fiverr.com###'
    link_attr = 'href'

    cards = soup.find_all(card_tag, class_= card_tag_class)
    cards_ids = []
    
    for card in cards:
        
        card_id = link_template.replace('###', card[link_attr])
        
        if len(card_id) > len(link_template):
            cards_ids.append(card_id)

    return  cards_ids

In [ ]:
def get_guru_links(job_type_: str) -> list:  

    soup = soups[soups['soup_name'] == 'guru_soup']['soup'][2]
    link_template = 'https://www.guru.com###'

    if job_type_ == 'freelancers':
        cards_parents = soup.find_all(
            'h2', class_= 'serviceListing__title serviceListing__title--dark')

        cards = []

        for parent in cards_parents:
            card = parent.find('a')
            cards.append(card)

    elif job_type_ == 'jobs':
        cards_parents = soup.find_all(
            'h2', class_ = 'jobRecord__title jobRecord__title--changeVisited')

        cards = []

        for parent in cards_parents:
            card = parent.find('a')
            cards.append(card)

    cards_ids = []

    for card in cards:
        card_id = link_template.replace('###', card['href'])
        cards_ids.append(card_id)

    return  cards_ids

## <center><strong> Collecting <span style= 'color: #517afc'>Cards</span> Data